#### improvement plan in it
#### 1.use dataset and dataloader class in it


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [4]:
df = pd.read_csv("/content/breast_cancer_data.csv")

In [5]:
df.head(2)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


In [6]:
df.drop(columns=['id','Unnamed: 32'],axis=1,inplace=True)

In [7]:
df.head(2)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [8]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2,random_state=0)

In [9]:
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)


In [10]:
lb = LabelEncoder()
y_train_scaled = lb.fit_transform(y_train)
y_test_scaled = lb.transform(y_test)

#### convert numpy array into tensor

In [11]:
X_train_tensor = torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test_scaled.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train_scaled.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test_scaled.astype(np.float32))


In [12]:
X_train_tensor.dtype

torch.float32

In [13]:
X_train_tensor.shape

torch.Size([455, 30])

In [14]:
y_train_tensor.shape

torch.Size([455])

In [15]:
y_train_tensor.dtype

torch.float32

In [16]:
from torch.utils.data import Dataset,DataLoader

class CustomDataset(Dataset):

  def __init__(self,features,labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self,index):

    return self.features[index],self.labels[index]

In [17]:
train_dataset = CustomDataset(X_train_tensor,y_train_tensor)
test_dataset = CustomDataset(X_test_tensor,y_test_tensor)

In [18]:
train_dataset[5]

(tensor([-0.2357, -1.3077, -0.2482, -0.3131, -0.9274, -0.3483, -0.4724, -0.5200,
         -0.7030, -0.9878, -0.7718, -1.2544, -0.6059, -0.5249, -0.9619, -0.2477,
         -0.1028, -0.5396, -0.7225, -0.4209, -0.4127, -1.3405, -0.3086, -0.4396,
         -0.7017,  0.1088,  0.0958, -0.3482, -0.2673, -0.5784]),
 tensor(0.))

In [19]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True)

#### define model

In [20]:
class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self ,features):
    out= self.linear(features)
    out = self.sigmoid(out)
    return out

  # def loss_function(self,y_pred,y):
  #   # clamp prediction to avoid log(0)
  #   epsilon = 1e-7
  #   y_pred = torch.clamp(y_pred,epsilon,1-epsilon)

  #   # calculate loss
  #   loss = - (y_train_tensor * torch.log(y_pred) +(1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
  #   return loss

In [21]:
learning_rate = 0.1
epochs = 25


In [22]:
# using built in loss function

loss_function = nn.BCELoss()

In [23]:
model = MySimpleNN(X_train_tensor.shape[1])

# using built in optimizer

optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

#### training pipeline

In [24]:


# define loop
for epoch in range(epochs):

  for batch_features,batch_labels in train_loader:

    # forward pass
    y_pred = model(X_train_tensor)

    # calculate loss

    loss = loss_function(y_pred,y_train_tensor.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # # parameter update

    optimizer.step()
    # # zero gardients
    # model.linear.weight.grad.zero_()
    # model.linear.bias.grad.zero_()

    # print loss in each epoch
    print(f'Epoch: {epoch + 1} , loss: {loss.item()}')

Epoch: 1 , loss: 0.6353493928909302
Epoch: 1 , loss: 0.5058535933494568
Epoch: 1 , loss: 0.42949095368385315
Epoch: 1 , loss: 0.3794717788696289
Epoch: 1 , loss: 0.34399116039276123
Epoch: 1 , loss: 0.31731435656547546
Epoch: 1 , loss: 0.2963743209838867
Epoch: 1 , loss: 0.27939319610595703
Epoch: 1 , loss: 0.26527079939842224
Epoch: 1 , loss: 0.25328925251960754
Epoch: 1 , loss: 0.24295899271965027
Epoch: 1 , loss: 0.23393407464027405
Epoch: 1 , loss: 0.22596226632595062
Epoch: 1 , loss: 0.2188548892736435
Epoch: 1 , loss: 0.21246765553951263
Epoch: 2 , loss: 0.20668800175189972
Epoch: 2 , loss: 0.2014266699552536
Epoch: 2 , loss: 0.19661174714565277
Epoch: 2 , loss: 0.1921846866607666
Epoch: 2 , loss: 0.1880970448255539
Epoch: 2 , loss: 0.18430858850479126
Epoch: 2 , loss: 0.18078529834747314
Epoch: 2 , loss: 0.17749838531017303
Epoch: 2 , loss: 0.1744232475757599
Epoch: 2 , loss: 0.17153868079185486
Epoch: 2 , loss: 0.16882629692554474
Epoch: 2 , loss: 0.16627009212970734
Epoch: 2 ,

In [25]:
model.linear.weight

Parameter containing:
tensor([[ 0.5668,  0.4619,  0.4255,  0.6006,  0.2640,  0.1222,  0.3927,  0.3780,
          0.1874, -0.1554,  0.7630, -0.0029,  0.5102,  0.4741,  0.0071, -0.3178,
         -0.1477,  0.0492, -0.0895, -0.4454,  0.6144,  0.7265,  0.7089,  0.7193,
          0.4064,  0.4020,  0.4762,  0.4972,  0.3735,  0.1836]],
       requires_grad=True)

In [26]:
model.linear.bias

Parameter containing:
tensor([-0.4008], requires_grad=True)

#### evaluations

In [28]:
# Model evaluation using test_loader

model.eval()
accuracy_list =[]
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    # forward pass
    y_pred = model(batch_features)
    y_pred = (y_pred > 0.8).float() # convert probabilities to binary predictions

    # calculate accuracy for the current batch
    batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
    accuracy_list.append(batch_accuracy)

# calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy : {overall_accuracy:.4f}')

Accuracy : 0.9627
